<a href="https://colab.research.google.com/github/HoangHungLN/MachineLearning_Assignment/blob/main/Extended_Assignment/notebooks/Extended_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import pandas as pd
import numpy as np
import json
import os
import getpass, os, subprocess, textwrap

os.makedirs("data", exist_ok = True)
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json -O data/train.json
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json -O data/dev.json
os.makedirs("modules", exist_ok= True)
!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/modules/forward_algorithm.py -O modules/forward_algorithm.py
!wget  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/modules/viterbi_algorithm.py -O modules/viterbi_algorithm.py

DATA_FILE = 'data/train.json'
DEV_FILE = 'data/dev.json'

--2025-11-12 14:18:03--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27164412 (26M) [text/plain]
Saving to: ‘data/train.json’

data/train.json     100%[===================>]  25.91M  --.-KB/s    in 0.1s    

2025-11-12 14:18:03 (271 MB/s) - ‘data/train.json’ saved [27164412/27164412]

--2025-11-12 14:18:03--  https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/dev.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133

In [89]:
with open(DATA_FILE, 'r', encoding='utf-8') as f:
    raw_train = json.load(f)
with open(DEV_FILE, 'r', encoding='utf-8') as f:
    raw_dev = json.load(f)

X_train = [ex['sentence'] for ex in raw_train]
Y_train = [ex['labels']     for ex in raw_train]
X_dev   = [ex['sentence'] for ex in raw_dev]
Y_dev   = [ex['labels']     for ex in raw_dev]

In [90]:
UNK = "<UNK>"
min_freq = 1

word_freq = Counter(w for sent in X_train for w in sent)
vocab = {w for w,c in word_freq.items() if c > min_freq}
vocab.add(UNK)
vocab = sorted(vocab)

tag_set = sorted({t for tags in Y_train for t in tags})

word2id = {w:i for i,w in enumerate(vocab)}
tag2id  = {t:i for i,t in enumerate(tag_set)}
id2tag  = {i:t for t,i in tag2id.items()}

def map_unk(sent):
    return [w if w in word2id else UNK for w in sent]

X_train = [map_unk(s) for s in X_train]
X_dev   = [map_unk(s) for s in X_dev]

In [91]:
import numpy as np

def estimate_hmm_supervised(X, Y, tag2id, word2id,
                            alpha_pi=1.0, alpha_A=1.0, alpha_B=1e-3):
    K, V = len(tag2id), len(word2id)
    pi_cnt = np.full(K,     alpha_pi, dtype=np.float64)
    A_cnt  = np.full((K,K), alpha_A,  dtype=np.float64)
    B_cnt  = np.full((K,V), alpha_B,  dtype=np.float64)

    for words, tags in zip(X, Y):
        if not words:
            continue
        pi_cnt[tag2id[tags[0]]] += 1
        for t in range(len(words)):
            j = tag2id[tags[t]]
            w = word2id[words[t]]
            B_cnt[j, w] += 1
            if t < len(words)-1:
                i = tag2id[tags[t]]
                k = tag2id[tags[t+1]]
                A_cnt[i, k] += 1

    pi = pi_cnt / pi_cnt.sum()
    A  = A_cnt / A_cnt.sum(axis=1, keepdims=True)
    B  = B_cnt / B_cnt.sum(axis=1, keepdims=True)
    return pi, A, B

pi, A, B = estimate_hmm_supervised(X_train, Y_train, tag2id, word2id)

#Giải thuật Forward


In [92]:
idx_unk = word2id[UNK]
results = []
for sentence_words in X_dev:
    # Bỏ qua câu rỗng (nếu có)
    if not sentence_words:
        continue

    # Chuyển câu sang chuỗi quan sát O (indices)
    O_indices = []
    for word in sentence_words:
        idx = word2id.get(word, idx_unk)
        O_indices.append(idx)
    O_indices = np.array(O_indices)

    prob = forward_algorithm(O_indices, pi, A, B)

    results.append( (' '.join(sentence_words), prob) )

results.sort(key=lambda x: x[1])

print(f"\n--- KẾT QUẢ THÍ NGHIỆM FORWARD (trên {len(results)} câu của dev.json) ---")

print(f"\n--- 10 CÂU CÓ XÁC SUẤT THẤP NHẤT ---")
# Lấy 10 câu đầu tiên (thấp nhất)
for sentence, p in results[:10]:
  print(f"Prob: {p: .2e} | Câu: {sentence}")

print(f"\n--- 10 CÂU CÓ XÁC SUẤT CAO NHẤT ---")
# Lấy 10 câu cuối (cao nhất) và lật ngược lại
for sentence, p in reversed(results[-10:]):
  print(f"Prob: {p: .2e} | Câu: {sentence}")


--- KẾT QUẢ THÍ NGHIỆM FORWARD (trên 5527 câu của dev.json) ---

--- 10 CÂU CÓ XÁC SUẤT THẤP NHẤT ---
Prob:  1.41e-246 | Câu: <UNK> <UNK> <UNK> , San Francisco , telecommunications holding company , annual sales of $ 9.5 billion , no damage to headquarters , but no power , the power failure has caused a delay in the release of the company 's earnings report , major concern is subsidiaries , Pacific Bell and Pacific Telesis Cellular , both of which sustained damage to buildings , structural damage to several cellular sites in Santa Cruz , volume of calls on cellular phones 10 times the usual , causing a big slowdown .
Prob:  3.93e-243 | Câu: Daniel von <UNK> is <UNK> but totally assured as Major Battle , <UNK> just the right brand of <UNK> and <UNK> ; Jeff Weiss is fire , <UNK> and <UNK> <UNK> as the <UNK> senator who serves as a friendly <UNK> of Major Battle ; <UNK> <UNK> is <UNK> <UNK> playing a succession of lawyers ; Joseph Daly has the perfect `` <UNK> , <UNK> '' <UNK> of George 

# Phân tích và Đánh giá Kết quả Giải thuật Forward

## 1. Các câu có xác suất cao nhất

10 câu có xác suất cao nhất đều là các câu cực kỳ ngắn (1-2 từ).Đáng chú ý nhất, các câu có xác suất cao tuyệt đối là "UNK ." và "UNK UNK".

Từ kết quả ta có thể thấy được hiện tượng thiên vị về độ dài. Cụ thể, các câu càng ngắn thì càng có xác suất lớn. Hiện tượng trên là do giải thuật Forward tính xác suất cuối cùng P(O) bằng cách nhân liên tiếp các xác suất tại mỗi bước thời gian.Vì tất cả các xác suất đều nhỏ hơn 1.0, về mặt toán học, câu càng ngắn thì càng ít phép nhân dẫn xác suất cuối cùng càng cao.

Bên cạnh đó, sự xuất hiện của UNK, UNK trở thành từ phổ biến nhất trong từ điển do quá trình thay thế từ hiếm với sự xuất hiện ít hơn 2 lần thành UNK, quá trình đó cộng dồn trong quá trình train tham số cho mô hình, dẫn đến UNK vô tình trở thành từ xuất hiện nhiều nhất, dẫn đến xác suất sinh ra UNK tại các trạng thái trở nên lớn hơn.

# 2. Các câu có xác suất thấp nhất
10 câu có xác suất thấp nhất đều là các câu rất dài (50–70+ từ), và chứa nhiều UNK cùng cấu trúc phức tạp.

Đây cũng là kết quả của hiện tượng thiên vị về độ dài, khi các từ có độ dài càng cao thì số lượng phép nhân cho số bé hơn 1 càng nhiều, dẫn đến giá trị xác suất trở nên rất bé.

Bên cạnh đó các câu này còn có cấu trúc ngữ pháp phức tạp hơn, khiến cho ma trận chuyển tiếp A chứa các giá trị chuyển trạng thái rất nhỏ khi gặp cấu trúc này.

# 3. Giá trị xác suất quá nhỏ
Đối với tập dữ liệu hiện tại, các câu có độ dài không quá lớn khiến cho giá trị xác suất không bị bé vượt ngưỡng e-324 của kiểu float64 mà numpy đang sử dụng. Nhưng với giải thuật hiện tại, nếu gặp câu có độ dài lớn hơn nhiều sẽ rất dễ gặp hiện tượng tràn số dưới dẫn đến giá trị xác suất trả về là 0.0. Để giải quyết cho hiện tượng trên, ta hoàn toàn có thể chuyển bài toán sang không gian logagite Bằng cách này, phép nhân xác suất (ví dụ: $P_1 \times P_2$) được chuyển đổi thành phép cộng log-probability (ví dụ: $\log(P_1) + \log(P_2)$), giúp tránh các giá trị bị làm tròn về 0. Khi cần thực hiện phép cộng trong không gian log, kỹ thuật "Log-Sum-Exp" sẽ được sử dụng để đảm bảo tính toán luôn ổn định.

# Kết quả
Mô hình HMM giải thích khá tốt về cấu trúc ngữ pháp cho câu, khi với độ dài nhất định, một câu có xác suất xảy ra cao hơn thì sẽ có cấu trúc ngữ pháp phù hợp hơn trong mô hình đang xét. Tuy nhiên, mô hình vẫn còn rất nhiều hạn chế như cách xử lý các từ hiểm quá thô sơ hay hiện tượng thiên vị độ dài.

Ta có thể hạn chế được điểm yếu trên bằng cách sử dụng chuẩn hóa xác suất của một chuỗi quan sát bằng cách chia cho độ dài câu. Hay cải tiến cách xử lý từ hiếm bằng phương pháp Subword Tokenization. Hoặc đơn giản hơn, ta sử dụng các mô hình hiện đại hơn và thông minh hơn trong việc xử lý chuỗi chẳng hạn như mạng Neuron.

# Giải thuật Viterbi
## Giới thiệu các nhãn POS (Penn Treebank)

Trong bài này, em dataset nhóm sử dụng bộ nhãn POS theo chuẩn **Penn Treebank**. Dưới đây là một số nhãn thường gặp:

### 1. Danh từ (Nouns)

| Tag   | Ý nghĩa                               | Ví dụ                        |
|-------|---------------------------------------|------------------------------|
| **NN**   | Danh từ thường, số ít                  | dog, house, book             |
| **NNS**  | Danh từ thường, số nhiều               | dogs, houses, books          |
| **NNP**  | Danh từ riêng, số ít                   | John, London, Tuesday        |
| **NNPS** | Danh từ riêng, số nhiều                | Americans, Europeans         |

### 2. Động từ (Verbs)

| Tag   | Ý nghĩa                                             | Ví dụ                              |
|-------|-----------------------------------------------------|------------------------------------|
| **VB**   | Động từ nguyên mẫu                                | eat, go, run                       |
| **VBD**  | Động từ quá khứ                                  | ate, went, ran                     |
| **VBG**  | Hiện tại phân từ / V-ing                         | eating, going, running            |
| **VBN**  | Quá khứ phân từ                                  | eaten, gone, broken               |
| **VBP**  | Hiện tại, không ngôi thứ 3 số ít                 | I eat, you go                      |
| **VBZ**  | Hiện tại, ngôi thứ 3 số ít                       | he eats, she goes                 |

### 3. Tính từ & Trạng từ

| Tag   | Ý nghĩa                         | Ví dụ                         |
|-------|---------------------------------|-------------------------------|
| **JJ**   | Tính từ                         | big, small, happy             |
| **JJR**  | Tính từ so sánh hơn             | bigger, smaller, happier      |
| **JJS**  | Tính từ so sánh nhất            | biggest, smallest, happiest   |
| **RB**   | Trạng từ                        | quickly, very, well           |
| **RBR**  | Trạng từ so sánh hơn            | faster, better                |
| **RBS**  | Trạng từ so sánh nhất           | fastest, best                 |

### 4. Đại từ, mạo từ, giới từ, liên từ

| Tag    | Ý nghĩa                           | Ví dụ                         |
|--------|-----------------------------------|--------------------------------|
| **PRP**   | Đại từ nhân xưng                  | I, you, he, she, they          |
| **PRP$**  | Đại từ sở hữu                     | my, your, his, her             |
| **DT**    | Mạo từ / từ hạn định              | a, an, the, this, those        |
| **IN**    | Giới từ / liên từ phụ thuộc       | in, on, at, of, because, if    |
| **CC**    | Liên từ đẳng lập                  | and, or, but                   |
| **TO**    | Từ *to* (trước động từ nguyên mẫu) | to go, to eat                  |

### 5. Một số nhãn khác

| Tag   | Ý nghĩa                     | Ví dụ                       |
|-------|-----------------------------|-----------------------------|
| **MD**   | Trợ động từ khuyết thiếu    | can, will, must, should     |
| **CD**   | Số từ                      |  10, 20       |
| **UH**   | Thán từ                    | oh,              |
| **. , : ; ? !** | Dấu câu            | . , : ; ? !                 |

Trong mô hình HMM, các nhãn POS ở trên chính là **trạng thái ẩn**, còn các từ trong câu là **chuỗi quan sát**. Nhiệm vụ của mô hình là, với mỗi câu đầu vào, tìm ra chuỗi nhãn POS phù hợp nhất cho từng từ.


In [93]:
states = tag_set

# Chuyển pi, A, B từ array sang dict mà hàm viterbi cần
start_p = {
    tag: float(pi[i])
    for i, tag in enumerate(tag_set)
}

trans_p = {
    tag_i: {
        tag_j: float(A[i, j])
        for j, tag_j in enumerate(tag_set)
    }
    for i, tag_i in enumerate(tag_set)
}

emit_p = {
    tag_i: {
        word: float(B[i, w])
        for w, word in enumerate(vocab)
    }
    for i, tag_i in enumerate(tag_set)
}

In [94]:
from modules.viterbi_algorithm import *
test_words = X_dev[1]
gold_tags  = Y_dev[1]

pred_tags, prob = viterbi_algorithm(test_words, states, start_p, trans_p, emit_p)

print("Câu test:")
print(test_words)
print("\nNhãn dự đoán:")
print(pred_tags)
print("\nNhãn gold:")
print(gold_tags)
print("\nXác suất:", prob)

Câu test:
['The', 'ruling', 'follows', 'a', 'host', 'of', 'problems', 'at', 'Tucson', 'Electric', ',', 'including', 'major', 'write-downs', ',', 'a', '60', '%', 'slash', 'in', 'the', 'common', 'stock', 'dividend', 'and', 'the', 'departure', 'of', 'former', 'Chairman', '<UNK>', '<UNK>', 'during', 'a', 'company', 'investigation', 'of', 'his', 'stock', 'sales', '.']

Nhãn dự đoán:
['DT', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'VBG', 'JJ', 'NNS', ',', 'DT', 'CD', 'NN', 'VB', 'IN', 'DT', 'JJ', 'NN', 'NN', 'CC', 'DT', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNP', 'IN', 'DT', 'NN', 'NN', 'IN', 'PRP$', 'NN', 'NNS', '.']

Nhãn gold:
['DT', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'NNS', 'IN', 'NNP', 'NNP', ',', 'VBG', 'JJ', 'NNS', ',', 'DT', 'CD', 'NN', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'CC', 'DT', 'NN', 'IN', 'JJ', 'NNP', 'NNP', 'NNP', 'IN', 'DT', 'NN', 'NN', 'IN', 'PRP$', 'NN', 'NNS', '.']

Xác suất: 1.2834557554191526e-108


In [95]:
raw_sent1 = ["he", "loves" "this", "subject", "the", "most"]
test_words1 = map_unk(raw_sent1)

pred_tags1, prob1 = viterbi_algorithm(test_words1, states, start_p, trans_p, emit_p)

print("Câu gốc:", raw_sent1)
print("Câu sau khi map UNK:", test_words1)
print("Nhãn dự đoán:", pred_tags1)
print("Xác suất:", prob1)

raw_sent2 = ["he", "is", "doing", "machine", "learning", "assignment"]
test_words2 = map_unk(raw_sent2)

pred_tags2, prob2 = viterbi_algorithm(test_words2, states, start_p, trans_p, emit_p)

print("Câu gốc:", raw_sent2)
print("Câu sau khi map UNK:", test_words2)
print("Nhãn dự đoán:", pred_tags2)
print("Xác suất:", prob2)

Câu gốc: ['he', 'lovesthis', 'subject', 'the', 'most']
Câu sau khi map UNK: ['he', '<UNK>', 'subject', 'the', 'most']
Nhãn dự đoán: ['PRP', 'VBZ', 'JJ', 'DT', 'RBS']
Xác suất: 1.367787998995333e-14
Câu gốc: ['he', 'is', 'doing', 'machine', 'learning', 'assignment']
Câu sau khi map UNK: ['he', 'is', 'doing', 'machine', 'learning', 'assignment']
Nhãn dự đoán: ['PRP', 'VBZ', 'VBG', 'NN', 'VBG', 'NN']
Xác suất: 9.515088851763234e-22


Khi thử một số câu tự tạo như:

- "he love this subject the most"
- "he is doing machine learning assignment"

mô hình gán nhãn khá hợp lý: phân biệt đúng các đại từ (PRP), động từ chia theo chủ ngữ (VBZ), dạng V-ing (VBG), mạo từ (DT), trạng từ so sánh nhất (RBS).

Các lỗi chủ yếu xuất hiện ở những cụm mơ hồ như "machine learning", nơi từ *learning* vừa có thể được gán là danh từ (NN) vừa có thể là động từ dạng V-ing (VBG). Đây là kiểu mơ hồ thường gặp của mô hình HMM sử dụng ngữ cảnh ngắn.


In [96]:
total_correct = 0
total_tokens  = 0

for words, gold in zip(X_dev, Y_dev):
    pred, _ = viterbi_algorithm(words, states, start_p, trans_p, emit_p)
    # đếm đúng / sai cho câu này
    for p, g in zip(pred, gold):
        if p == g:
            total_correct += 1
        total_tokens += 1

overall_acc = total_correct / total_tokens
print("Accuracy toàn dev set:", overall_acc)

Accuracy toàn dev set: 0.9482120089854896


### **Nhận xét kết quả**

Đoạn code trên tính **độ chính xác (accuracy)** trên tập dev theo công thức:

$$
\text{accuracy} = \frac{\text{số token gán đúng nhãn}}{\text{tổng số token}}
$$

Kết quả thu được:

- **Accuracy trên dev ≈ 0.9482 (≈ 94.8%)**

Như vậy, với một mô hình HMM rất cơ bản (ước lượng tham số bằng đếm tần suất có smoothing, dùng token `<UNK>` cho từ hiếm) và thuật toán Viterbi, hệ thống đã gán đúng POS cho gần **95% số từ** trong tập kiểm tra. Đây là một kết quả khá tốt đối với HMM thuần túy, cho thấy mô hình đã học được phân bố:

- xác suất bắt đầu câu với từng nhãn,
- xác suất chuyển tiếp giữa các nhãn (A),
- xác suất phát xạ từ ứng với từng nhãn (B).

Phần **~5% còn lại** chủ yếu rơi vào các trường hợp mơ hồ về từ loại, ví dụ những từ vừa có thể là danh từ vừa có thể là động từ/tính từ, hoặc các từ ít xuất hiện trong tập huấn luyện. Đây là giới hạn tự nhiên của HMM bậc 1 chỉ dùng thông tin ngữ cảnh rất ngắn (tag ngay trước), chưa tận dụng được ngữ cảnh xa hay thông tin hình thái (suffix, viết hoa, v.v.).

Trong các hướng phát triển tiếp theo, có thể cải thiện bằng cách:
- thêm đặc trưng cho từ mới (đuôi `-ing`, `-ed`, số nhiều `-s`, chữ hoa…),
- dùng mô hình mạnh hơn như BiLSTM-CRF hoặc Transformer-based tagger,